In [11]:
from tqdm import tqdm
from pathlib import Path
import shutil
from sklearn.decomposition import PCA
import numpy as np

layer_num = 12
variance_ratio = 0.9
model = "Wav2Vec2"
# dir_path = "/content/drive/MyDrive/Grad project/spk_phone_joint_matrices/" + model + "/layer_" + str(layer_num) # path to speaker matrix
dir_path = "/content/drive/MyDrive/Grad project/spk_phone_joint_matrices/" + model + "/train-clean-100-layer12"
speaker_arrays = np.load(dir_path + "/speaker_matrix.npy")
pca_speaker = PCA(n_components= variance_ratio, svd_solver='full')
principal_components_speaker = pca_speaker.fit_transform(speaker_arrays) # remove transpose if (40,512) shape
input_files = list(Path("/content/drive/MyDrive/Grad project/models_hidden_states/" + model + "/test-clean").rglob("*.npy")) # CPC representations directory

FileNotFoundError: ignored

In [ ]:

import os
dataset = "train-clean-100"
# dir_path = "/content/drive/MyDrive/Grad project/models_hidden_states/Wav2Vec2/layers_avg/layer_" + str(layer_num)
output_path = "/content/drive/MyDrive/Grad project/models_hidden_states/" + model + "/" + "test-clean" + "-layer" + str(layer_num) + "_collapsed_" + str(variance_ratio) +"_on_"+dataset
os.makedirs(output_path, exist_ok=True)

In [ ]:
print(output_path)

/content/drive/MyDrive/Grad project/models_hidden_states/Hubert_ASR/test-clean-layer12_collapsed_0.95_on_train-clean-100


In [10]:
print(len(pca_speaker.explained_variance_ratio_))

81


In [ ]:


for path in tqdm(input_files):
  frames_collapsed = []
  frames = np.load(path, mmap_mode="r")[layer_num].squeeze()
  # frames = np.load(path, mmap_mode="r")
  for frame in frames:
    z = frame
    for pca in pca_speaker.components_:
      # z = z - (np.transpose(z) * pca) * pca
      z = np.subtract(z, np.multiply(np.matmul(z, pca), pca))
    # pca = np.transpose(principal_components_speaker)[0]
    # z = z - (((np.transpose(z) * pca) * pca) * 0.1)

    frames_collapsed.append(z)
  file_name = str(path).split("/")[-1]
  file_name = file_name.split(".")[0]
  output_frames = np.array(frames_collapsed)
  np.save(output_path + "/" + file_name + ".npy", output_frames)
  # shutil.copy(path, "/content/drive/MyDrive/Grad project/PCA/orig_one_list")

100%|██████████| 2620/2620 [34:42<00:00,  1.26it/s]


In [ ]:
frames.shape

(1043, 768)